# Bin Fill Frequency

**USE CASE - 1:**

Analytics for frequency of filling of particular smart bin. Time duration of filled smart bins. Means how much time takes to fill a bin. Needs to monitor frequent fill bin level.

Source Table: swmnetbins_agg_data

Target Table: swm_netbins_binfill_frequency

Programming Logic:

For each of every 43 smart bins installed across NDMC, the data for the smart bin is stored in the swmnetbins_agg_data table. The Fill Rate value is updated for each bin based on the historical fill data of that bin. This Fill rate is value gives us an idea how much percentage of the bin will get filled in an hour. We process this data to calculate the number of hours it would take for the bin to get to 100% full based on the fill rate. This gives the end user an idea how frequently the bin will get filled and the time it would take to fill the bin. This Information is stored in the target table: swm_netbins_binfill_frequency, in intervals of every half day. The schedulers are scheduled to run once every day to update the latest information.

In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [2]:
def df_info_func(df, vizualize=False, threshold=3, display_sample=True, side_by_side=True):
    def f1(series):
        try: return str(series.unique().tolist())
        except: return '--'

    print(f'Rows: {df.shape[0]} N Cols: {df.shape[1]}')

    if df.shape[0] > 0:
        df_info = pd.DataFrame(index=df.columns)
        df_info['data_types'] = df.dtypes.values
        df_info['n_missing'] = df.isna().sum().values    
        df_info['missing_pct'] = round((df_info['n_missing'] / len(df))*100,2)

        df_ = df.astype(str).copy()

        df_info['n_unique'] = df_.apply(lambda x: x.nunique(),axis=0).values
        df_info['uniq_vals'] = df_.apply(lambda x: np.where(x.nunique() <= threshold, f1(x), '--')).values

        df_info.reset_index(inplace=True)
        df_info.rename(columns={'index':'cols'},inplace=True)
        if vizualize:
            # import seaborn as sns
            # sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')
            # or an another package to display missing values graph
            import missingno as msno 
            msno.matrix(df)

        if side_by_side:
            from IPython.display import display_html
            space = "\xa0" * 10
            info_styler = df_info.style.set_table_attributes("style='display:inline'").set_caption('DF Info')
            df_styler = df.head().style.set_table_attributes("style='display:inline'").set_caption('DF')
            display_html(info_styler._repr_html_() + space + df_styler._repr_html_(),raw=True)            
        else:
            from IPython.core.display import HTML
            display(HTML(df_info.to_html()))
            if display_sample: display(HTML(df.head().to_html()))
            
    else:
        print("Empty DataFrame")

In [3]:
import os
import sys
import json
import requests
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
warnings.filterwarnings('ignore')

time_now = datetime.now().strftime("%Y-%m-%d %H:%M")
today_date = datetime.now().strftime("%Y-%m-%d")

api_csv_col_name_mapping = {"entity_id": "entity_id", "sid": "identifier", "deviceid": "device_id",
                            "latitude": "latitude", "longitude": "longitude", "location": "location",
                            "circle": "circle", "bintype": "bin_type", "temperature": "temperature",
                            "battery": "battery", "signallevel": "signallevel", "fillelevel": "filledlevel",
                            "fillrate": "fill_rate","iterationsarray": "iterations_array",
                            "pickedfilllevel": "picked_filllevel", "transactions": "transactions",
                            "filltime": "fill_time", "sourcetimestamp": "source_timestamp",
                            "last_updated":"last_updated", "pickedat": "picked_at",
                            "created_at": "created_at", "updated_at": "update_timestamp"}

date_columns = ['source_timestamp','last_updated','picked_at','created_at','update_timestamp']


class DataRetrivalApi():
    
    """
        This Class Fetches the data from the data from the API
        for the table swm_netbins_agg_data from NDMC 2.x db.
    """

    def __init__(self):
        self.auth_url = "https://dashboard.ndmc.com/ds/1.0.0/public/token"
        self.auth_headers = {'Content-Type': 'application/json'}
        self.auth_payload = json.dumps({"username": "kartheek@ndmc.com",
                                        "password": "Mullapudi@1",
                                        "grant_type": "password"})

        self.data_url = "https://dashboard.ndmc.com/abstraction/1.0.0/dashboard/getData/U8-eC3sBwOPaHTv4EWJP"
        self.data_payload = {"from": "2019-01-01", "to": today_date}
        self.data_headers = {'Accept': 'application/json, text/plain, */*',
                             'Authorization': 'Bearer {access_token}',
                             'Content-Type': 'application/json',
                             'Origin': 'https://dashboard.ndmc.com'}

    def authenticate(self):
        response = requests.request("POST",
                                    self.auth_url, headers=self.auth_headers,
                                    data=self.auth_payload, verify=False)
        if response.status_code == 200:
            access_token = json.loads(response.text)['access_token']
            return access_token
        else:
            print("Couldn't Authenticate API Credentials")
            print("Status Code: ", response.status_code)
            print("Status Text: ", response.text)
            raise Exception("SourceAPIAuthenticationError")

    def fetch_data(self, from_date=None, till_date=None):
        # print(self.data_payload)
        data_payload = self.data_payload
        if from_date:
            data_payload['from'] = from_date
        if till_date:
            data_payload['to'] = till_date

        # print(data_payload)
        data_payload = json.dumps(data_payload)
        # print(data_payload)

        access_token = self.authenticate()
        data_headers = self.data_headers
        data_headers['Authorization'] = data_headers['Authorization'].format(access_token=access_token)

        response = requests.request("POST", self.data_url,headers=self.data_headers,
                                    data=data_payload, verify=False)

        if response.status_code == 200:
            results = json.loads(response.text)
            if len(results['result']) > 0:
                df = pd.DataFrame(results['result'])
                return df
            else:
                print("No Records Found In the Given Time Period")
                return None
        else:
            print("Error While Fetching Data From Source API")
            print("Status Code: ", response.status_code)
            print("Status Code: ", response.text)
            raise Exception("SourceAPIDataFetchError")
            

def get_swm_api_df():
    d_ret = DataRetrivalApi()
    swm_netbins_df = d_ret.fetch_data()
    swm_netbins_df.rename(columns=api_csv_col_name_mapping, inplace=True)   
    
    for d in date_columns:        
        try:
            swm_netbins_df[d] = pd.to_datetime(swm_netbins_df[d],format='%Y-%m-%d %H:%M:%S')
            # .strftime("%Y-%m-%d")
            # .strftime("%Y-%m-%d %H:%M:%S")
            # ,format='%Y-%m-%d %H:%M:%S'
        except:
            pass
    swm_netbins_df.sort_values(by=['source_timestamp', 'location', 'identifier'], inplace=True)
    swm_netbins_df.drop_duplicates(subset=['source_timestamp', 'location', 'identifier'], inplace=True, ignore_index=True)
    return swm_netbins_df

def get_daterange_helper(df):
    min_date_value = pd.to_datetime(df.source_timestamp.dt.date.min().strftime('%Y-%m-%d 00:00:00'))
    max_date_value = pd.to_datetime(df.source_timestamp.dt.date.max().strftime('%Y-%m-%d 23:59:59'))
    each_hour_date_list = pd.date_range(min_date_value, max_date_value, freq='12H')
    return each_hour_date_list

def process_bin_fill_frequency_for_a_location(df):

    temp_binfill_frequency_list = []
    if len(df) > 0:
        every_halfday_list = get_daterange_helper(df)
        bin_location = df.location.unique().tolist()[0]
        # print(every_halfday_list)
        
        for a_half_day in every_halfday_list:
            # print(type(a_half_day))
            filtered_df = df.loc[((df.source_timestamp <= a_half_day.strftime("%Y-%m-%d %H:%M:%S")) & (~df.fill_rate.isna())),]
            if len(filtered_df) > 0:
                fill_rate_value = filtered_df.loc[filtered_df.source_timestamp == filtered_df.source_timestamp.max(), 'fill_rate'].values[-1]
                binfill_frequency_dict = {'location': bin_location,
                                          'identifier': location_identifier_mapping[bin_location],
                                          'source_timestamp': a_half_day,
                                          'fill_rate': fill_rate_value,
                                          'fillrate_last_updated': filtered_df.source_timestamp.max()}
                temp_binfill_frequency_list.append(binfill_frequency_dict)

    return temp_binfill_frequency_list

def hours_to_fill_helper(x):
    try:
        return round(100 / x, 2)
    except ZeroDivisionError:
        return 0


def bin_fill_frequency_data(source_data=None):
    if not source_data:
        source_data = get_swm_api_df()

    source_mapping = source_data.set_index('location').identifier.to_dict()
    global location_identifier_mapping
    location_identifier_mapping = source_data.set_index('location').identifier.to_dict()

    binfill_frequency_list = []
    for bin_location in source_mapping.keys():
        # print("Processing: ",bin_location)
        source_location_data = source_data.loc[source_data.location == bin_location]
        bin_fills = process_bin_fill_frequency_for_a_location(source_location_data)
        binfill_frequency_list.extend(bin_fills)

    if len(binfill_frequency_list) > 0:
        df = pd.DataFrame(binfill_frequency_list)
        df['fill_rate'].fillna(value=0,inplace=True)
        df['fill_rate'] = df['fill_rate'].astype(float).astype(int)
        df['hours_to_fill'] = df['fill_rate'].apply(lambda x: hours_to_fill_helper(x))
    else:
        df = pd.DataFrame()

    return df

# Fetch Source Data 

In [4]:
source_data = get_swm_api_df()
df_info_func(source_data)

Rows: 1168 N Cols: 25


,cols,data_types,n_missing,missing_pct,n_unique,uniq_vals
0,entity_id,object,0,0.000000,43,--
1,identifier,object,0,0.000000,43,--
2,provider,object,0,0.000000,1,['NDMC SWM Netbins']
3,version,object,0,0.000000,1,['V1.0']
4,geo_address,object,0,0.000000,27,--
5,last_updated,"datetime64[ns, UTC]",0,0.000000,923,--
6,created_at,"datetime64[ns, UTC]",0,0.000000,1161,--
7,update_timestamp,"datetime64[ns, UTC]",0,0.000000,1168,--
8,device_id,object,0,0.000000,43,--
9,latitude,object,0,0.000000,26,--


In [5]:
source_data.sort_values(by=['location','source_timestamp'],inplace=True)
latest_fill_data = source_data.drop_duplicates(subset=['location'],keep='last')[['location','source_timestamp','filledlevel','fill_rate']].reset_index(drop=True)

def hours_to_fill_helper(x):
    try:
        return round(100 / x, 2)
    except ZeroDivisionError:
        return 0

latest_fill_data['fill_rate'].fillna(value=0,inplace=True)
latest_fill_data['fill_rate'] = latest_fill_data['fill_rate'].astype(float).astype(int)
latest_fill_data['filledlevel'] = latest_fill_data['filledlevel'].astype(float).astype(int)    
latest_fill_data['hours_to_fill'] = latest_fill_data['fill_rate'].apply(lambda x: hours_to_fill_helper(x))

latest_fill_data['source_timestamp'] = latest_fill_data['source_timestamp'].dt.tz_localize(None)
latest_fill_data['source_timestamp_rounded'] = latest_fill_data['source_timestamp'].dt.round('H')

df_info_func(latest_fill_data)
latest_fill_data.head()

Rows: 43 N Cols: 6


,cols,data_types,n_missing,missing_pct,n_unique,uniq_vals
0,location,object,0,0.000000,43,--
1,source_timestamp,datetime64[ns],0,0.000000,39,--
2,filledlevel,int32,0,0.000000,31,--
3,fill_rate,int32,0,0.000000,23,--
4,hours_to_fill,float64,0,0.000000,23,--
5,source_timestamp_rounded,datetime64[ns],0,0.000000,26,--
,location,source_timestamp,filledlevel,fill_rate,hours_to_fill,source_timestamp_rounded
0,Bengali Market 1,2020-09-16 10:09:00,0,0,0.000000,2020-09-16 10:00:00
1,Bengali Market 2,2020-06-01 18:16:00,0,0,0.000000,2020-06-01 18:00:00
2,Delhi Golf Club 1,2020-08-22 18:21:00,0,0,0.000000,2020-08-22 18:00:00


,location,source_timestamp,filledlevel,fill_rate,hours_to_fill,source_timestamp_rounded
0,Bengali Market 1,2020-09-16 10:09:00,0,0,0.00,2020-09-16 10:00:00
1,Bengali Market 2,2020-06-01 18:16:00,0,0,0.00,2020-06-01 18:00:00
2,Delhi Golf Club 1,2020-08-22 18:21:00,0,0,0.00,2020-08-22 18:00:00
3,Himachal Bhavan 3,2021-08-06 14:10:00,70,36,2.78,2021-08-06 14:00:00
4,Himachal Bhavan 4,2021-08-06 14:04:00,20,15,6.67,2021-08-06 14:00:00


In [21]:
max_date_value = pd.to_datetime('now').round('H') + pd.Timedelta(hours=24)

new_list = []
alfa_list = []

    for row_index, col_dict in latest_fill_data.iterrows():

        location = col_dict['location']
        fill_rate = col_dict['fill_rate']
        hours_to_fill = col_dict['hours_to_fill']
        present_filledlevel = col_dict['filledlevel']

        min_date_value = pd.to_datetime(col_dict['source_timestamp_rounded'])
        each_hour_date_list = pd.date_range(min_date_value, max_date_value, freq='H')

        hrs_since_update = int(
            (pd.to_datetime('now').round('H') - col_dict['source_timestamp_rounded']) / pd.Timedelta(hours=1))

        alfa_one_list = []
        if fill_rate != 0:
            f_level = present_filledlevel
            for an_hour in each_hour_date_list[1:]:
                f_level += fill_rate
                temp_dict = {
                    'location': location,
                    'last_update_timestamp': min_date_value.strftime("%Y-%m-%d %H:%M:%S"),
                    'last_update_fillevel': present_filledlevel,
                    'last_update_fillrate': fill_rate,
                    'hours_since_last_update': hrs_since_update,
                    'forecast_timestamp': an_hour,
                    'forecast_fillevel': min(100, f_level)}
                new_list.append(temp_dict)

                if an_hour.round("D") >= pd.to_datetime('now').round('D'):
                    alfa_one_dict = {'forecast_timestamp': an_hour.strftime("%Y-%m-%d %H:%M:%S"),
                                     'forecast_fillevel': min(100, f_level)}
                    alfa_one_list.append(alfa_one_dict)
        else:
            for an_hour in each_hour_date_list[1:]:
                temp_dict = {
                    'location': location,
                    'last_update_timestamp': min_date_value.strftime("%Y-%m-%d %H:%M:%S"),
                    'last_update_fillevel': present_filledlevel,
                    'last_update_fillrate': fill_rate,
                    'hours_since_last_update': hrs_since_update,
                    'forecast_timestamp': an_hour,
                    'forecast_fillevel': present_filledlevel}
                new_list.append(temp_dict)

                if an_hour.round("D") >= pd.to_datetime('now').round('D'):
                    alfa_one_dict = {'forecast_timestamp': an_hour.strftime("%Y-%m-%d %H:%M:%S"),
                                     'forecast_fillevel': present_filledlevel}
                    alfa_one_list.append(alfa_one_dict)

        alfa_two_dict = {'location': location,
                         'last_update_timestamp': min_date_value.strftime("%Y-%m-%d %H:%M:%S"),
                         'last_update_fillevel': present_filledlevel,
                         'last_update_fillrate': fill_rate,
                         'hours_since_last_update': hrs_since_update,
                         'forecasts': alfa_one_list}
        alfa_list.append(alfa_two_dict)

    new_df = pd.DataFrame(new_list)
    new_df = new_df[new_df.forecast_timestamp.dt.date >= pd.to_datetime('now').round('D')]

print(new_df.shape)
new_df = new_df[new_df.forecast_timestamp.dt.date >= pd.to_datetime('now').round('D')]
print(new_df.shape)
new_df.head()

(69900, 7)
(1253, 7)


,location,last_update_timestamp,last_update_fillevel,last_update_fillrate,hours_since_last_update,forecast_timestamp,forecast_fillevel
7765,Bengali Market 1,2020-09-16 10:00:00,0,0,7777,2021-08-06 00:00:00,0
7766,Bengali Market 1,2020-09-16 10:00:00,0,0,7777,2021-08-06 01:00:00,0
7767,Bengali Market 1,2020-09-16 10:00:00,0,0,7777,2021-08-06 02:00:00,0
7768,Bengali Market 1,2020-09-16 10:00:00,0,0,7777,2021-08-06 03:00:00,0
7769,Bengali Market 1,2020-09-16 10:00:00,0,0,7777,2021-08-06 04:00:00,0


In [27]:
# json.dumps(alfa_list)